In [ ]:
import json
import pennylane as qml
import pennylane.numpy as np

# Write any helper functions you need here


dev = qml.device('default.qubit', wires=[0,1,2])

@qml.qnode(dev)
def cloning_machine(coefficients, wire):
    
    """
    Returns the reduced density matrix on a wire for the cloning machine circuit.
    
    Args:
        - coefficients (np.array(float)): an array [c0,c1] containing the coefficients parametrizing
        the input state fed into the middle and bottom wires of the cloning machine.
        wire (int): The wire on which we calculate the reduced density matrix.

    Returns:
        - np.tensor(complex): The reduced density matrix on wire = wire, as returned by qml.density_matrix.
    
    """
    
    c0, c1 = coefficients

    # Put your code here
    #qml.StatePrep(np.array([(c0+c1)/np.sqrt(2), c1/np.sqrt(2), 0, c0/np.sqrt(2)]), wires=[1, 2])
    
    coefficients = coefficients / np.sqrt(2)
    c0 = coefficients[0]
    c1 = coefficients[1]
    theta1 = 2 * np.arcsin(c0)
    theta2 = np.arcsin(c1 / (np.cos(theta1 / 2)))

    # circuit can create 3 states 00 01 and 11,
    # doing those operations we find evaluations for the states weights
    # and after angles of the rotation
    qml.RY(theta1, 1)
    qml.RY(theta2, 2)
    qml.CNOT([1, 2])
    qml.RY(theta2, 2)

    qml.CNOT([0,1])
    qml.CNOT([0,2])
    qml.CNOT([1,0])
    qml.CNOT([2,0])
    
    # Return the reduced density matrix
    return qml.density_matrix(wires=wire)


def fidelity(coefficients):
    
    """
    Calculates the fidelities between the reduced density matrices in wires 0 and 1 and the input state |0>.
    
    Args:
        - coefficients (np.array(float)): an array [c0,c1] containing the coefficients parametrizing
        the input state fed into the middle and bottom wires of the cloning machine.
    Returns:
        - (np.array(float)): An array whose elements are:
            - 0th element:  The fidelity between the output reduced state on wire 0 and the state |0>.
            - 1st element:  The fidelity between the output reduced state on wire 1 and the state |0>.    
    """
    


    # Put your code here
    q1 = cloning_machine(coefficients, wire=0)
    q2 = cloning_machine(coefficients, wire=1)
    print("q1: ", q1)
    print("q2: ", q2)
    zero_state = np.array([[1,0],[0,0]])
    
    f1 = qml.math.fidelity(zero_state, q1)
    f2 = qml.math.fidelity(zero_state, q2)
    print("f1: ", f1)
    print("f2: ", f2)
    return np.array([f1, f2])


# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = fidelity(ins).tolist()
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    u = cloning_machine([1/np.sqrt(3),1/np.sqrt(3)],1)
    for op in cloning_machine.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CNOT)), "You are using forbidden gates!"
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct fidelities"


# These are the public test cases
test_cases = [
    ('[0.5773502691896258, 0.5773502691896257]', '[0.83333333, 0.83333333]'),
    ('[0.2, 0.8848857801796105]', '[0.60848858, 0.98]')
]

# This will run the public test cases locally
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

## Attempts with symbolic linear algebra

In [22]:
from sympy import Matrix, Symbol, pi, sqrt, cos, sin, acos

# Define the RY gate matrix
def RY(alpha):
    return Matrix([[cos(alpha/2), -sin(alpha/2)], [sin(alpha/2), cos(alpha/2)]])

# Define the qubit vector representing |0⟩ state
qubit_0 = Matrix([[1], [0]])
qubit_00 = qubit_0 * qubit_0.T
print(qubit_00)

# Define symbolic parameters c0 and c1
c0, c1 = Symbol('c0'), Symbol('c1')

# Compute the value of alpha
alpha_value = 2 * acos((c0 + c1) / sqrt(2))

# Apply the RY gate to the first qubit
result_state = RY(alpha_value) * qubit_00

# Reshape the resulting state to obtain the vector representation
result_vector = result_state.reshape(4, 1)

# Display the results
print("RY gate matrix:")
print(RY(alpha_value))
print("\n|00⟩ state vector:")
print(qubit_00)
print("\nComputed value of alpha:")
print(alpha_value)
print("\nResulting state vector after applying RY gate to the first qubit:")
print(result_vector)


Matrix([[1, 0], [0, 0]])
RY gate matrix:
Matrix([[sqrt(2)*(c0 + c1)/2, -sqrt(1 - (c0 + c1)**2/2)], [sqrt(1 - (c0 + c1)**2/2), sqrt(2)*(c0 + c1)/2]])

|00⟩ state vector:
Matrix([[1, 0], [0, 0]])

Computed value of alpha:
2*acos(sqrt(2)*(c0 + c1)/2)

Resulting state vector after applying RY gate to the first qubit:
Matrix([[sqrt(2)*(c0 + c1)/2], [0], [sqrt(1 - (c0 + c1)**2/2)], [0]])


In [29]:
from sympy import Matrix, Symbol, pi, sqrt, cos, sin, acos, asin

# Define the RY gate matrix
def RY(alpha, qubit_index):
    # Identity matrix for a two-qubit system
    identity_matrix = Matrix.eye(4)

    # RY gate matrix for the specified qubit
    ry_matrix = Matrix([[cos(alpha/2), -sin(alpha/2)], [sin(alpha/2), cos(alpha/2)]])

    # Place the RY gate matrix on the corresponding qubit in the 4x4 identity matrix
    gate_matrix = identity_matrix.reshape(4,4)
    gate_matrix = gate_matrix.reshape(4, 4)
    gate_matrix[:, :] = identity_matrix
    gate_matrix[qubit_index * 2 : (qubit_index + 1) * 2, qubit_index * 2 : (qubit_index + 1) * 2] = ry_matrix

    return gate_matrix

# Define the qubit vector representing |00⟩ state
qubit_0 = Matrix([[1], [0], [0], [0]])

# Define symbolic parameters c0 and c1
c0, c1 = Symbol('c0'), Symbol('c1')

# Compute the value of alpha
alpha_value = 2 * acos((c0 + c1) / sqrt(2))

# Apply the RY gate to the first qubit
state_after_first_ry = RY(alpha_value, 0) * qubit_0

# Compute the value of beta
beta_value = asin(c1 / (sin(alpha_value / 2) * sqrt(2)))

# Apply the RY gate to the second qubit
result_state = RY(beta_value, 1) * state_after_first_ry

# Display the results
print("RY gate matrix for alpha:")
print(RY(alpha_value, 0))
print("\n|00⟩ state vector:")
print(qubit_0)
print("\nComputed value of alpha:")
print(alpha_value)
print("\nRY gate matrix for beta:")
print(RY(beta_value, 1))
print("\nComputed value of beta:")
print(beta_value)
print("\nResulting state vector after applying RY gates:")
print(result_state)


RY gate matrix for alpha:
Matrix([[sqrt(2)*(c0 + c1)/2, -sqrt(1 - (c0 + c1)**2/2), 0, 0], [sqrt(1 - (c0 + c1)**2/2), sqrt(2)*(c0 + c1)/2, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

|00⟩ state vector:
Matrix([[1], [0], [0], [0]])

Computed value of alpha:
2*acos(sqrt(2)*(c0 + c1)/2)

RY gate matrix for beta:
Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), -sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)], [0, 0, sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)]])

Computed value of beta:
asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))

Resulting state vector after applying RY gates:
Matrix([[sqrt(2)*(c0 + c1)/2], [sqrt(1 - (c0 + c1)**2/2)], [0], [0]])


In [87]:
from sympy import Matrix, Symbol, pi, sqrt, cos, sin, acos, asin, simplify

# Define the RY gate matrix
"""
def RY(alpha, qubit_index):
    # Identity matrix for a two-qubit system
    identity_matrix = Matrix.eye(4)

    # RY gate matrix for the specified qubit
    ry_matrix = Matrix([[cos(alpha/2), -sin(alpha/2)], [sin(alpha/2), cos(alpha/2)]])

    # Place the RY gate matrix on the corresponding qubit in the 4x4 identity matrix
    gate_matrix = identity_matrix.reshape(4,4)
    gate_matrix = gate_matrix.reshape(4, 4)
    if qubit_index == 0:
        #gate_matrix[:, :] = identity_matrix
        #gate_matrix[qubit_index * 2 : (qubit_index + 1) * 2, qubit_index * 2 : (qubit_index + 1) * 2] = ry_matrix
        gate_matrix = Matrix([[cos(alpha/2), -sin(alpha/2), 0, 0],
                              [sin(alpha/2), cos(alpha/2), 0, 0],
                              [0, 0, 1, 0],
                              [0, 0, 0, 1]])
        #gate_matrix = Matrix([[1, 0, 0, 0],
        #                      [0, 1, 0, 0],
        #                      [0, 0, cos(alpha/2), -sin(alpha/2)],
        #                      [0, 0, sin(alpha/2), cos(alpha/2)]])
        #gate_matrix = Matrix([[cos(alpha/2), 0, -sin(alpha/2), 0],
        #                      [0, 1, 0, 0],
        #                      [sin(alpha/2), 0, cos(alpha/2), 0],
        #                      [0, 0, 0, 1]])
    if qubit_index == 1:
        gate_matrix = Matrix([[1, 0, 0, 0],
                              [0, cos(alpha/2), 0, -sin(alpha/2)],
                              [0, 0, 1, 0],
                              [0, sin(alpha/2), 0, cos(alpha/2)]])

    return gate_matrix
"""

def RY(alpha, qubit_index):
    # Identity matrix for a two-qubit system
    identity_matrix = eye(2)

    # RY gate matrix for the specified qubit
    ry_matrix = Matrix([[cos(alpha/2), -sin(alpha/2)],
                       [sin(alpha/2), cos(alpha/2)]])

    # Create the Kronecker product with the identity matrix for the other qubit
    if qubit_index == 0:
        gate_matrix = TensorProduct(ry_matrix, identity_matrix)
    elif qubit_index == 1:
        gate_matrix = TensorProduct(identity_matrix, ry_matrix)
    else:
        raise ValueError("Invalid qubit index")

    return gate_matrix

# Define the CNOT gate matrix
def CNOT(control, target):
    # Identity matrix for a two-qubit system
    #identity_matrix = Matrix.eye(4)

    # CNOT gate matrix
    #cnot_matrix = identity_matrix.reshape(4,4)
    #cnot_matrix = cnot_matrix.reshape(4, 4)
    # Define the correct CNOT matrix depending on the control and target qubits
    # Define the correct CNOT matrix depending on the control and target qubits
    if control == 0 and target == 1:
        cnot_matrix = Matrix([[1, 0, 0, 0],
                              [0, 1, 0, 0],
                              [0, 0, 0, 1],
                              [0, 0, 1, 0]])
    elif control == 1 and target == 0:
        cnot_matrix = Matrix([[1, 0, 0, 0],
                              [0, 0, 0, 1],
                              [0, 0, 1, 0],
                              [0, 1, 0, 0]])
    else:
        # Invalid control and target qubits for a CNOT gate
        raise ValueError("Invalid control and target qubits for CNOT gate")
    
    return cnot_matrix

# Define the qubit vector representing |00⟩ state
qubit_0 = Matrix([[1], [0], [0], [0]])

# Define symbolic parameters c0 and c1
c0, c1 = Symbol('c0'), Symbol('c1')

# Compute the value of alpha
alpha_value = 2 * acos(c0+c1)
# Compute the value of beta
beta_value = asin(c1 / (sin(alpha_value / 2)))

# Apply the RY gate to the first qubit
state_after_first_ry = RY(alpha_value, 0) * qubit_0



result0 = RY(alpha_value, 0) * qubit_0
result1 = RY(beta_value, 1) * result0
result2 = CNOT(0, 1) * result1
result3 = RY(beta_value, 1) * result2

#result = CNOT(0,1) * CNOT(1,0) * CNOT(0,1) * RY(-1*beta_value, 1) * CNOT(0, 1) * RY(beta_value, 1) * RY(alpha_value, 0) * qubit_0

# Display the results
print("RY gate matrix for alpha:")
print(RY(alpha_value, 0))
print("\n|00⟩ state vector:")
print(qubit_0)
print("\nComputed value of alpha:")
print(alpha_value)
print("\nComputed value of beta:")
print(beta_value)
print("\nResults:")
print("\nafter RY alpha")
print(result0)
print("\nafter RY beta")
print(result1)
print("\nafter CNOT")
print(result2)
print("\nafter RY beta")
print(result3)


RY gate matrix for alpha:
Matrix([[c0 + c1, 0, -sqrt(1 - (c0 + c1)**2), 0], [0, c0 + c1, 0, -sqrt(1 - (c0 + c1)**2)], [sqrt(1 - (c0 + c1)**2), 0, c0 + c1, 0], [0, sqrt(1 - (c0 + c1)**2), 0, c0 + c1]])

|00⟩ state vector:
Matrix([[1], [0], [0], [0]])

Computed value of alpha:
2*acos(c0 + c1)

Computed value of beta:
asin(c1/sqrt(1 - (c0 + c1)**2))

Results:

after RY alpha
Matrix([[c0 + c1], [0], [sqrt(1 - (c0 + c1)**2)], [0]])

after RY beta
Matrix([[(c0 + c1)*cos(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [(c0 + c1)*sin(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [sqrt(1 - (c0 + c1)**2)*cos(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [sqrt(1 - (c0 + c1)**2)*sin(asin(c1/sqrt(1 - (c0 + c1)**2))/2)]])

after CNOT
Matrix([[(c0 + c1)*cos(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [(c0 + c1)*sin(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [sqrt(1 - (c0 + c1)**2)*sin(asin(c1/sqrt(1 - (c0 + c1)**2))/2)], [sqrt(1 - (c0 + c1)**2)*cos(asin(c1/sqrt(1 - (c0 + c1)**2))/2)]])

after RY beta
Matrix([[-(c0 + c1)*sin(asin(c1/sqrt(1 - 

In [63]:
print(CNOT(0,1)*RY(-c0,1)*CNOT(0,1)*RY(c0,1))

Matrix([[1, 0, 0, 0], [0, cos(c0/2)**2, sin(c0/2), -sin(c0/2)*cos(c0/2)], [0, -sin(c0/2)*cos(c0/2), cos(c0/2), sin(c0/2)**2], [0, sin(c0/2), 0, cos(c0/2)]])


In [62]:
print(CNOT(0,1) * CNOT(1,0) * CNOT(0,1) * RY(-1*beta_value, 1) * CNOT(0, 1) * RY(beta_value, 1) * RY(alpha_value, 0))

Matrix([[sqrt(2)*(c0 + c1)/2, -sqrt(1 - (c0 + c1)**2/2), 0, 0], [sqrt(1 - (c0 + c1)**2/2)*sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), sqrt(2)*(c0 + c1)*sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)/2, 0, cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)], [sqrt(1 - (c0 + c1)**2/2)*cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)**2, sqrt(2)*(c0 + c1)*cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)**2/2, sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), -sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)*cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)], [-sqrt(1 - (c0 + c1)**2/2)*sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)*cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), -sqrt(2)*(c0 + c1)*sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)*cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)/2, cos(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2), sin(asin(sqrt(2)*c1/(2*sqrt(1 - (c0 + c1)**2/2)))/2)**2]])


In [80]:
import numpy as np

def RY_matrix(theta):
    return np.array([[np.cos(theta/2), -np.sin(theta/2)],
                     [np.sin(theta/2), np.cos(theta/2)]])

# Rotation around Y-axis on the First Qubit
theta1 = 0.5  # Example angle
RY_1 = np.kron(RY_matrix(theta1), np.eye(2))

# Rotation around Y-axis on the Second Qubit
theta2 = 0.3  # Example angle
RY_2 = np.kron(np.eye(2), RY_matrix(theta2))

print("RY matrix for the First Qubit:")
print(RY_1)

print("\nRY matrix for the Second Qubit:")
print(RY_2)

RY matrix for the First Qubit:
[[ 0.96891242  0.         -0.24740396 -0.        ]
 [ 0.          0.96891242 -0.         -0.24740396]
 [ 0.24740396  0.          0.96891242  0.        ]
 [ 0.          0.24740396  0.          0.96891242]]

RY matrix for the Second Qubit:
[[ 0.98877108 -0.14943813  0.         -0.        ]
 [ 0.14943813  0.98877108  0.          0.        ]
 [ 0.         -0.          0.98877108 -0.14943813]
 [ 0.          0.          0.14943813  0.98877108]]


In [85]:
from sympy import symbols, cos, sin, eye, Matrix
from sympy.physics.quantum import TensorProduct

def RY(alpha, qubit_index):
    # Identity matrix for a two-qubit system
    identity_matrix = eye(2)

    # RY gate matrix for the specified qubit
    ry_matrix = Matrix([[cos(alpha/2), -sin(alpha/2)],
                       [sin(alpha/2), cos(alpha/2)]])

    # Create the Kronecker product with the identity matrix for the other qubit
    if qubit_index == 0:
        gate_matrix = TensorProduct(ry_matrix, identity_matrix)
    elif qubit_index == 1:
        gate_matrix = TensorProduct(identity_matrix, ry_matrix)
    else:
        raise ValueError("Invalid qubit index")

    return gate_matrix

# Example usage:
alpha_value = symbols('alpha')
RY_matrix_0 = RY(alpha_value, 0)
RY_matrix_1 = RY(alpha_value, 1)

print("RY matrix for the First Qubit:")
print(RY_matrix_0)

print("\nRY matrix for the Second Qubit:")
print(RY_matrix_1)

RY matrix for the First Qubit:
Matrix([[cos(alpha/2), 0, -sin(alpha/2), 0], [0, cos(alpha/2), 0, -sin(alpha/2)], [sin(alpha/2), 0, cos(alpha/2), 0], [0, sin(alpha/2), 0, cos(alpha/2)]])

RY matrix for the Second Qubit:
Matrix([[cos(alpha/2), -sin(alpha/2), 0, 0], [sin(alpha/2), cos(alpha/2), 0, 0], [0, 0, cos(alpha/2), -sin(alpha/2)], [0, 0, sin(alpha/2), cos(alpha/2)]])
